In [1]:
import os
import glob
import time
import json
import pandas as pd
from multiprocessing import Pool
import multiprocessing as mp

from parse_json_param import *
from _full_train_minute_price import *
import numpy as np


Using TensorFlow backend.


In [2]:
def run_process(param):
    DATA_PARAMS, MODEL_PARAMS, TARGET_TO_PREDICT = param
    try:
        DATA_PARAMS, MODEL_PARAMS = initialize_path_parameters(DATA_PARAMS, MODEL_PARAMS, TARGET_TO_PREDICT)
        full_train_minute_price(DATA_PARAMS, MODEL_PARAMS)
        return [1, TARGET_TO_PREDICT, "ok", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    except Exception as err:
        return [0, TARGET_TO_PREDICT, err, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    
def run_one_setup(json_setup_file):
    p = load_param_json(json_setup_file)
    DATA_PARAMS, MODEL_PARAMS, possible_assets, num_instances = parse_raw_param(p)
    runs_params = [(DATA_PARAMS, MODEL_PARAMS, TARGET_TO_PREDICT) for TARGET_TO_PREDICT in possible_assets]

    t0 = time.time()
    pool = Pool(processes=num_instances)
    run_status = pool.map(run_process, runs_params)
    pool.close()
    pool.join()

    #completion_df
    completion_df = pd.DataFrame(run_status, columns=["status","target","err", "completion_time"]).set_index("target")
    completion_df.to_csv(os.path.join("json_setup", "CompletionSummary", os.path.basename(json_setup_file).split(".")[0] + ".csv"))

    #move parameter to done
    shutil.move(json_setup_file, os.path.join("json_setup", "Done"))

    t1 = time.time()

    print("Setup Done,", json_setup_file, ". Time Elapsed:", t1-t0, "seconds.")


In [3]:

def run_by_gpu(json_files, gpu):
    for json_setup_file in json_files:
        device = load_param_json(json_setup_file)["MODEL_PARAMS"]["device"]
        if device == gpu:
            run_one_setup(json_setup_file)
    




In [4]:
json_files = glob.glob(os.path.join("json_setup", "*.json"))
json_and_device = dict()
json_and_device["json"] = []
json_and_device["device"] = []
for json_setup_file in json_files:
    p = load_param_json(json_setup_file)
    json_and_device["json"].append(json_setup_file)
    json_and_device["device"].append(p["MODEL_PARAMS"]["device"])

In [5]:
jobs = []
for gpu in np.unique(json_and_device["device"]):
    p = mp.Process(target=run_by_gpu, args=(json_files,gpu))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()
    

EURUSD
EWZ
IAU
====== PREPROCESSING =======
====== PREPROCESSING =======
VWO
====== PREPROCESSING =======
XLI
====== PREPROCESSING =======
====== PREPROCESSING =======
join
XLI
====== PREPROCESSING =======
EWZ
====== PREPROCESSING =======
VWO
EURUSD
====== PREPROCESSING =======
====== PREPROCESSING =======
IAU
====== PREPROCESSING =======
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!
Load Data: Done!


Process ForkPoolWorker-2:3:
Traceback (most recent call last):
Process ForkPoolWorker-2:5:
Process ForkPoolWorker-2:4:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-2:1:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process ForkPoolWorker-1:2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Process ForkPoolWorker-2:2:
  File "<ipython-input-2-6d6bd122e30d>", line 5, in run_process
    full_train_minute_price(DATA_PARAMS, MODEL_PARAMS)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._tar

KeyboardInterrupt: 

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-e0ee199435f6>", line 6, in run_by_gpu
    run_one_setup(json_setup_file)
  File "<ipython-input-2-6d6bd122e30d>", line 17, in run_one_setup
    run_status = pool.map(run_process, runs_params)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 266, in map
    return self._map_async(func, iterable, mapstar, chunksize).get()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 638, in get
    self.wait(timeout)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 635, in wait
    self._event.wait(timeout)
  File "/usr/lib/python3.6/threading.py", line 551, in wait
    signaled = self._cond.wait(timeout)
  File "/usr/lib/python3.6/threading.py", line 295, in wait
    waiter.acquire()
Ke